In [1]:
import numpy as np
import os
import shutil
import subprocess



In [2]:
total_count = 16


replica_count = 4

n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [3]:
# regions = [ 'us-west1-b','us-west1-b', \
#            'europe-west3-c', 'europe-west3-c', \
#            'asia-south1-c', 'asia-south1-c']

# regions = [ 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b']
regions = [ 'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']


# regions = ['europe-west3-c',  \
#            'asia-south1-c']


n_regions = len(regions)

In [4]:
for i,k in enumerate(regions):
    print(i,k)

0 us-west1-b
1 us-west1-b
2 us-west1-b
3 us-west1-b


In [5]:
lines  = []
zones = {}


lim = 0
for reg in regions:
    for i in range(replica_count+int(nclients/n_clusters)):
        line = 'gcloud compute instances create resdb-'+f"{lim+i:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
        
        print(line)
        lines.append(line)
        zones[i] = reg
    
    lim = lim + replica_count+int(nclients/n_clusters)
        
    
    
    
with open('launch_cloud_instances.sh', 'w') as f:
    for item in lines:
        f.write(str(item) + '\n')


gcloud compute instances create resdb-00 --project=ucr-ursa-major-lesani-lab --zone=us-west1-b --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=g

In [6]:
import subprocess
import concurrent.futures

def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=len(commands)) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-19].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-00].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-06].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-03].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-07].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-02].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-14].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-04].
Created [https://www.googleapis.com/comp

NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-00  us-west1-b  e2-highmem-2               10.138.0.20  35.230.19.169  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP    STATUS
resdb-19  us-west1-b  e2-highmem-2               10.138.0.111  34.82.193.151  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP   STATUS
resdb-06  us-west1-b  e2-highmem-2               10.138.0.104  34.83.10.223  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.



NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP    STATUS
resdb-03  us-west1-b  e2-highmem-2               10.138.0.106  34.145.94.181  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP     STATUS
resdb-02  us-west1-b  e2-highmem-2               10.138.0.113  35.203.165.111  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP  STATUS
resdb-14  us-west1-b  e2-highmem-2               10.138.0.109  34.83.235.4  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
resdb-04  us-west1-b  e2-highmem-2               10.138.0.85  34.83.24.149  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
resdb-15  us-west1-b  e2-highmem-2               10.138.0.28  34.83.158.26  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.



...

NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP   STATUS
resdb-09  us-west1-b  e2-highmem-2               10.138.0.112  35.197.45.21  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.

 - The res

NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP   STATUS
resdb-10  us-west1-b  e2-highmem-2               10.138.0.117  34.168.244.5  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP   STATUS
resdb-16  us-west1-b  e2-highmem-2               10.138.0.114  34.83.77.160  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP    STATUS
resdb-01  us-west1-b  e2-highmem-2               10.138.0.105  35.197.49.196  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP      STATUS
resdb-12  us-west1-b  e2-highmem-2               10.138.0.115  104.198.105.249  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP    STATUS
resdb-05  us-west1-b  e2-highmem-2               10.138.0.110  34.168.19.127  RUNNING
NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP     STATUS
resdb-17  us-west1-b  e2-highmem-2               10.138.0.1

 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20231023'.



NAME      ZONE        MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP    STATUS
resdb-11  us-west1-b  e2-highmem-2               10.138.0.118  35.197.47.114  RUNNING


In [7]:
# os.system('sh launch_cloud_instances.sh')

In [8]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [9]:
# os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


instance_type = 'e2-small'


# for index, region  in enumerate(regions[::4]):
# for index, region  in enumerate(regions[::2]):
for index, region  in enumerate(regions[::4]):

# for index, region  in enumerate(regions):
    
    print(index, region)
    redirect = '>>'
    if index==0:
        redirect = '>'
        
    os.system('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
              redirect+' all_internal_ips')
    
    
    print('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
              redirect+' all_internal_ips')


    os.system('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_external_ips')
    
    
    print('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_external_ips')
    
    










0 us-west1-b
gcloud compute instances list --filter="ZONE=us-west1-b" --format="value(networkInterfaces[0].networkIP)" > all_internal_ips
gcloud compute instances list --filter="ZONE=us-west1-b" --format="value(networkInterfaces[0].networkIP)" > all_external_ips


In [10]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [11]:
ipList, len(ipList)

(['10.138.0.20',
  '10.138.0.105',
  '10.138.0.113',
  '10.138.0.106',
  '10.138.0.85',
  '10.138.0.110',
  '10.138.0.104',
  '10.138.0.108',
  '10.138.0.9',
  '10.138.0.112',
  '10.138.0.117',
  '10.138.0.118',
  '10.138.0.115',
  '10.138.0.107',
  '10.138.0.109',
  '10.138.0.28',
  '10.138.0.114',
  '10.138.0.116',
  '10.138.0.26',
  '10.138.0.111'],
 20)

In [12]:
clusters_per_region_map = {}

clusters_per_region_map[0] = 0
clusters_per_region_map[1] = 0
clusters_per_region_map[2] = 0

n_regions = 1

for i in range(n_clusters):
    
    
    clusters_per_region_map[i%n_regions] = clusters_per_region_map[i%n_regions] + 1

In [13]:
clusters_per_region_map

{0: 4, 1: 0, 2: 0}

In [14]:
new_server_only_ips= []
new_client_only_ips= []


current_region_total = 0
for i in range(n_regions):
    print(current_region_total,current_region_total+ clusters_per_region_map[i] *replica_count)
    new_server_only_ips = new_server_only_ips + ipList[current_region_total:current_region_total+ clusters_per_region_map[i] *replica_count]
    
    new_client_only_ips = new_client_only_ips + ipList[current_region_total+ clusters_per_region_map[i] *replica_count:
                                                      current_region_total+ clusters_per_region_map[i] *(replica_count+1)]
    
    
    current_region_total = current_region_total + clusters_per_region_map[i] *replica_count +  clusters_per_region_map[i]
    
    

0 16


In [15]:
new_server_only_ips, len(new_server_only_ips)

(['10.138.0.20',
  '10.138.0.105',
  '10.138.0.113',
  '10.138.0.106',
  '10.138.0.85',
  '10.138.0.110',
  '10.138.0.104',
  '10.138.0.108',
  '10.138.0.9',
  '10.138.0.112',
  '10.138.0.117',
  '10.138.0.118',
  '10.138.0.115',
  '10.138.0.107',
  '10.138.0.109',
  '10.138.0.28'],
 16)

In [16]:
new_client_only_ips, len(new_client_only_ips) 

(['10.138.0.114', '10.138.0.116', '10.138.0.26', '10.138.0.111'], 4)

In [17]:
ipList = new_server_only_ips + new_client_only_ips

In [18]:
ipList, len(ipList)

(['10.138.0.20',
  '10.138.0.105',
  '10.138.0.113',
  '10.138.0.106',
  '10.138.0.85',
  '10.138.0.110',
  '10.138.0.104',
  '10.138.0.108',
  '10.138.0.9',
  '10.138.0.112',
  '10.138.0.117',
  '10.138.0.118',
  '10.138.0.115',
  '10.138.0.107',
  '10.138.0.109',
  '10.138.0.28',
  '10.138.0.114',
  '10.138.0.116',
  '10.138.0.26',
  '10.138.0.111'],
 20)

In [19]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]




clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(10000)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(20000)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(10000)
    clusterPortMap2[7001+i] = str(20000)



In [20]:
clusterPortMap1, clusterPortMap2

({0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  8: '10000',
  9: '10000',
  10: '10000',
  11: '10000',
  12: '10000',
  13: '10000',
  14: '10000',
  15: '10000',
  7001: '10000',
  7002: '10000',
  7003: '10000',
  7004: '10000'},
 {0: '20000',
  1: '20000',
  2: '20000',
  3: '20000',
  4: '20000',
  5: '20000',
  6: '20000',
  7: '20000',
  8: '20000',
  9: '20000',
  10: '20000',
  11: '20000',
  12: '20000',
  13: '20000',
  14: '20000',
  15: '20000',
  7001: '20000',
  7002: '20000',
  7003: '20000',
  7004: '20000'})

In [21]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 

Starting a Gradle Daemon (subsequent builds will be faster)

> Task :compileJava


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 12s
3 actionable tasks: 3 executed


0

In [22]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

0

In [23]:
os.system('echo 3108 | sudo -S ./gradlew clean')

[sudo] password for tejas: 

> Task :clean

BUILD SUCCESSFUL in 996ms
1 actionable task: 1 executed


0

In [24]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

7003

----------

7004

----------



In [25]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [26]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

7001 10.138.0.114 10000

7002 10.138.0.116 10000

7003 10.138.0.26 10000

7004 10.138.0.111 10000



In [27]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [28]:
replica_count

4

In [29]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

f is 1
f is 1
f is 1
f is 1


In [30]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

config0/keysRSA/publickey7002 config0/keysRSA/publickey1000
config0/keysRSA/publickey7002 config0/keysRSA/publickey1001
config0/keysRSA/publickey7002 config0/keysRSA/publickey1002
config0/keysRSA/publickey7002 config0/keysRSA/publickey1003
config0/keysRSA/publickey7002 config0/keysRSA/publickey1004
config0/keysRSA/publickey7002 config0/keysRSA/publickey1005
config0/keysRSA/publickey7002 config0/keysRSA/publickey1006
config0/keysRSA/publickey7002 config0/keysRSA/publickey1007
config0/keysRSA/publickey7002 config0/keysRSA/publickey1008
config0/keysRSA/publickey7002 config0/keysRSA/publickey1009
config0/keysRSA/publickey7002 config0/keysRSA/publickey1010
config0/keysRSA/publickey7002 config0/keysRSA/publickey1011
config0/keysRSA/publickey7002 config0/keysRSA/publickey1012
config0/keysRSA/publickey7002 config0/keysRSA/publickey1013
config0/keysRSA/publickey7002 config0/keysRSA/publickey1014
config0/keysRSA/publickey7002 config0/keysRSA/publickey1015
config0/keysRSA/publickey7002 config0/ke

config0/keysRSA/publickey7002 config0/keysRSA/publickey1338
config0/keysRSA/publickey7002 config0/keysRSA/publickey1339
config0/keysRSA/publickey7002 config0/keysRSA/publickey1340
config0/keysRSA/publickey7002 config0/keysRSA/publickey1341
config0/keysRSA/publickey7002 config0/keysRSA/publickey1342
config0/keysRSA/publickey7002 config0/keysRSA/publickey1343
config0/keysRSA/publickey7002 config0/keysRSA/publickey1344
config0/keysRSA/publickey7002 config0/keysRSA/publickey1345
config0/keysRSA/publickey7002 config0/keysRSA/publickey1346
config0/keysRSA/publickey7002 config0/keysRSA/publickey1347
config0/keysRSA/publickey7002 config0/keysRSA/publickey1348
config0/keysRSA/publickey7002 config0/keysRSA/publickey1349
config0/keysRSA/publickey7002 config0/keysRSA/publickey1350
config0/keysRSA/publickey7002 config0/keysRSA/publickey1351
config0/keysRSA/publickey7002 config0/keysRSA/publickey1352
config0/keysRSA/publickey7002 config0/keysRSA/publickey1353
config0/keysRSA/publickey7002 config0/ke

config1/keysRSA/publickey7002 config1/keysRSA/publickey1075
config1/keysRSA/publickey7002 config1/keysRSA/publickey1076
config1/keysRSA/publickey7002 config1/keysRSA/publickey1077
config1/keysRSA/publickey7002 config1/keysRSA/publickey1078
config1/keysRSA/publickey7002 config1/keysRSA/publickey1079
config1/keysRSA/publickey7002 config1/keysRSA/publickey1080
config1/keysRSA/publickey7002 config1/keysRSA/publickey1081
config1/keysRSA/publickey7002 config1/keysRSA/publickey1082
config1/keysRSA/publickey7002 config1/keysRSA/publickey1083
config1/keysRSA/publickey7002 config1/keysRSA/publickey1084
config1/keysRSA/publickey7002 config1/keysRSA/publickey1085
config1/keysRSA/publickey7002 config1/keysRSA/publickey1086
config1/keysRSA/publickey7002 config1/keysRSA/publickey1087
config1/keysRSA/publickey7002 config1/keysRSA/publickey1088
config1/keysRSA/publickey7002 config1/keysRSA/publickey1089
config1/keysRSA/publickey7002 config1/keysRSA/publickey1090
config1/keysRSA/publickey7002 config1/ke

config2/keysRSA/publickey7002 config2/keysRSA/publickey1055
config2/keysRSA/publickey7002 config2/keysRSA/publickey1056
config2/keysRSA/publickey7002 config2/keysRSA/publickey1057
config2/keysRSA/publickey7002 config2/keysRSA/publickey1058
config2/keysRSA/publickey7002 config2/keysRSA/publickey1059
config2/keysRSA/publickey7002 config2/keysRSA/publickey1060
config2/keysRSA/publickey7002 config2/keysRSA/publickey1061
config2/keysRSA/publickey7002 config2/keysRSA/publickey1062
config2/keysRSA/publickey7002 config2/keysRSA/publickey1063
config2/keysRSA/publickey7002 config2/keysRSA/publickey1064
config2/keysRSA/publickey7002 config2/keysRSA/publickey1065
config2/keysRSA/publickey7002 config2/keysRSA/publickey1066
config2/keysRSA/publickey7002 config2/keysRSA/publickey1067
config2/keysRSA/publickey7002 config2/keysRSA/publickey1068
config2/keysRSA/publickey7002 config2/keysRSA/publickey1069
config2/keysRSA/publickey7002 config2/keysRSA/publickey1070
config2/keysRSA/publickey7002 config2/ke

config2/keysRSA/publickey7002 config2/keysRSA/publickey1199
config2/keysRSA/publickey7002 config2/keysRSA/publickey1200
config2/keysRSA/publickey7002 config2/keysRSA/publickey1201
config2/keysRSA/publickey7002 config2/keysRSA/publickey1202
config2/keysRSA/publickey7002 config2/keysRSA/publickey1203
config2/keysRSA/publickey7002 config2/keysRSA/publickey1204
config2/keysRSA/publickey7002 config2/keysRSA/publickey1205
config2/keysRSA/publickey7002 config2/keysRSA/publickey1206
config2/keysRSA/publickey7002 config2/keysRSA/publickey1207
config2/keysRSA/publickey7002 config2/keysRSA/publickey1208
config2/keysRSA/publickey7002 config2/keysRSA/publickey1209
config2/keysRSA/publickey7002 config2/keysRSA/publickey1210
config2/keysRSA/publickey7002 config2/keysRSA/publickey1211
config2/keysRSA/publickey7002 config2/keysRSA/publickey1212
config2/keysRSA/publickey7002 config2/keysRSA/publickey1213
config2/keysRSA/publickey7002 config2/keysRSA/publickey1214
config2/keysRSA/publickey7002 config2/ke

config3/keysRSA/publickey7002 config3/keysRSA/publickey1110
config3/keysRSA/publickey7002 config3/keysRSA/publickey1111
config3/keysRSA/publickey7002 config3/keysRSA/publickey1112
config3/keysRSA/publickey7002 config3/keysRSA/publickey1113
config3/keysRSA/publickey7002 config3/keysRSA/publickey1114
config3/keysRSA/publickey7002 config3/keysRSA/publickey1115
config3/keysRSA/publickey7002 config3/keysRSA/publickey1116
config3/keysRSA/publickey7002 config3/keysRSA/publickey1117
config3/keysRSA/publickey7002 config3/keysRSA/publickey1118
config3/keysRSA/publickey7002 config3/keysRSA/publickey1119
config3/keysRSA/publickey7002 config3/keysRSA/publickey1120
config3/keysRSA/publickey7002 config3/keysRSA/publickey1121
config3/keysRSA/publickey7002 config3/keysRSA/publickey1122
config3/keysRSA/publickey7002 config3/keysRSA/publickey1123
config3/keysRSA/publickey7002 config3/keysRSA/publickey1124
config3/keysRSA/publickey7002 config3/keysRSA/publickey1125
config3/keysRSA/publickey7002 config3/ke

config3/keysRSA/publickey7002 config3/keysRSA/publickey1249
config3/keysRSA/publickey7002 config3/keysRSA/publickey1250
config3/keysRSA/publickey7002 config3/keysRSA/publickey1251
config3/keysRSA/publickey7002 config3/keysRSA/publickey1252
config3/keysRSA/publickey7002 config3/keysRSA/publickey1253
config3/keysRSA/publickey7002 config3/keysRSA/publickey1254
config3/keysRSA/publickey7002 config3/keysRSA/publickey1255
config3/keysRSA/publickey7002 config3/keysRSA/publickey1256
config3/keysRSA/publickey7002 config3/keysRSA/publickey1257
config3/keysRSA/publickey7002 config3/keysRSA/publickey1258
config3/keysRSA/publickey7002 config3/keysRSA/publickey1259
config3/keysRSA/publickey7002 config3/keysRSA/publickey1260
config3/keysRSA/publickey7002 config3/keysRSA/publickey1261
config3/keysRSA/publickey7002 config3/keysRSA/publickey1262
config3/keysRSA/publickey7002 config3/keysRSA/publickey1263
config3/keysRSA/publickey7002 config3/keysRSA/publickey1264
config3/keysRSA/publickey7002 config3/ke

In [31]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [37]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

[main 61b793e5] Working with modified ClusterInfo
 3 files changed, 19355 insertions(+), 12894 deletions(-)


To github.com:tmane002/BftSmart_Final.git
   8ce8b6e3..61b793e5  main -> main


0

In [33]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [34]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any